$\beta = -1$ means that $\beta$ is selected at random in each iteration.

In [2]:
from grasp_iters import read_results

# iters = read_results("rl1323_882_441_0")
iters = read_results("pr1002_668_334_4")

In [3]:
iters

,iter,beta,RGD_OF,AFVS_OF,time,is_new_best
0,0,0.983703,5328,3034,0.872687,True
1,1,0.467560,3099,3034,1.077572,False
2,2,0.835733,4301,3034,1.675252,False
3,3,0.476608,3639,3034,2.002868,False
4,4,0.465620,4301,3034,2.347064,False
...,...,...,...,...,...,...
4995,4995,0.750435,3639,3034,1614.197964,False
4996,4996,0.802851,4915,3034,1614.625928,False
4997,4997,0.754179,3959,3034,1615.070404,False
4998,4998,0.893271,5162,3034,1615.654401,False


In [4]:
iters[iters["is_new_best"] == True]

,iter,beta,RGD_OF,AFVS_OF,time,is_new_best
0,0,0.983703,5328,3034,0.872687,True
